In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Cloning the YOLOv5 repository


In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12219, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 12219 (delta 6), reused 11 (delta 4), pack-reused 12197
Receiving objects: 100% (12219/12219), 12.61 MiB | 1.43 MiB/s, done.
Resolving deltas: 100% (8410/8410), done.


#Installing Requirements

In [3]:
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 8.1 MB/s 


#
Installing albumnetations

In [4]:
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install albumentations==1.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 4.9 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [6]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 9.4 MB/s 
     |████████████████████████████████| 158 kB 63.6 MB/s 
     |████████████████████████████████| 181 kB 74.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 157 kB 68.0 MB/s 
     |████████████████████████████████| 157 kB 57.8 MB/s 
     |████████████████████████████████| 157 kB 73.6 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 157 kB 76.2 MB/s 
     |████████████████████████████████| 157 kB 78.6 MB/s 
     |████████████████████████████████| 157 kB 78.9 MB/s 
     |████████████████████████████████| 156 kB 77.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=68b8b4bd940ac7da9cee6887f6593d587102407c8ba6304bde869685fa71537b
  Stored in directory: /root/.cache/pip/wheels/3e/31/

#Creating Model architecture configuration file

In [7]:
with open('new_train_yaml', 'w+') as file:
    file.write(
        """
        # parameters
        nc: 3  # number of classes
        depth_multiple: 0.33  # model depth multiple
        width_multiple: 0.50  # layer channel multiple

        # anchors
        anchors:
          - [10,13, 16,30, 33,23]  # P3/8
          - [30,61, 62,45, 59,119]  # P4/16
          - [116,90, 156,198, 373,326]  # P5/32

        # YOLOv5 backbone
        backbone:
          # [from, number, module, args]
          [[-1, 1, Focus, [64, 3]],  # 0-P1/2
           [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
           [-1, 3, BottleneckCSP, [128]],
           [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
           [-1, 9, BottleneckCSP, [256]],
           [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
           [-1, 9, BottleneckCSP, [512]],
           [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
           [-1, 1, SPP, [1024, [5, 9, 13]]],
           [-1, 3, BottleneckCSP, [1024, False]],  # 9
          ]

        # YOLOv5 head
        head:
          [[-1, 1, Conv, [512, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 6], 1, Concat, [1]],  # cat backbone P4
           [-1, 3, BottleneckCSP, [512, False]],  # 13

           [-1, 1, Conv, [256, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 4], 1, Concat, [1]],  # cat backbone P3
           [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

           [-1, 1, Conv, [256, 3, 2]],
           [[-1, 14], 1, Concat, [1]],  # cat head P4
           [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

           [-1, 1, Conv, [512, 3, 2]],
           [[-1, 10], 1, Concat, [1]],  # cat head P5
           [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

           [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
          ]
        """
    )

#Unzip dataset

In [9]:
import zipfile
from google.colab import drive
zip_ref = zipfile.ZipFile("/content/FaceMaskDetectionyolo.zip", 'r')
zip_ref.extractall("/content/Data2")
zip_ref.close()

#Creating training configuration file

In [35]:
with open('new_data_yaml', 'w+') as file:
    file.write(
        """
        train: /content/Data2/train
        val: /content/Data2/valid

        nc: 3
        names: ['mask_weared_incorrect','with_mask','without_mask'
]
        """
    )

#Training Our Custom Object Detector Model

In [36]:
!python /content/yolov5/train.py --img 416 --batch 64 --epochs 200 --data /content/new_data_yaml --cfg /content/new_train_yaml  --weights yolov5s.pt

wandb: Currently logged in as: faisal007. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=/content/new_train_yaml, data=/content/new_data_yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=64, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-99-g3cd66b1 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias

In [26]:
!python /content/yolov5/train.py --resume

wandb: Currently logged in as: faisal007. Use `wandb login --relogin` to force relogin
train: weights=yolov5/yolov5s.pt, cfg=, data=yolov5/data/coco128.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-99-g3cd66b1 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.

In [27]:
!zip -r /content/yolov5result2.zip /content/yolov5

  adding: content/yolov5/ (stored 0%)
  adding: content/yolov5/requirements.txt (deflated 50%)
  adding: content/yolov5/.gitignore (deflated 55%)
  adding: content/yolov5/train.py (deflated 69%)
  adding: content/yolov5/classify/ (stored 0%)
  adding: content/yolov5/classify/train.py (deflated 66%)
  adding: content/yolov5/classify/predict.py (deflated 68%)
  adding: content/yolov5/classify/val.py (deflated 65%)
  adding: content/yolov5/__pycache__/ (stored 0%)
  adding: content/yolov5/__pycache__/val.cpython-37.pyc (deflated 43%)
  adding: content/yolov5/__pycache__/export.cpython-37.pyc (deflated 51%)
  adding: content/yolov5/LICENSE (deflated 66%)
  adding: content/yolov5/.git/ (stored 0%)
  adding: content/yolov5/.git/FETCH_HEAD (deflated 67%)
  adding: content/yolov5/.git/description (deflated 14%)
  adding: content/yolov5/.git/hooks/ (stored 0%)
  adding: content/yolov5/.git/hooks/fsmonitor-watchman.sample (deflated 53%)
  adding: content/yolov5/.git/hooks/applypatch-msg.sample (

In [28]:
from google.colab import files
files.download('/content/yolov5result2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt --source /content/FaceMask32.mp4

detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/FaceMask32.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-99-g3cd66b1 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
new_train_yaml summary: 232 layers, 7251912 parameters, 0 gradients
video 1/1 (1/294) /content/FaceMask32.mp4: 384x640 4 with_masks, 12.9ms
video 1/1 (2/294) /content/FaceMask32.mp4: 384x640 4 with_masks, 9.3ms
video 1/1 (3/294) /content/FaceMask32.mp4: 384x640 4 with_masks, 9.9ms
video 1/1 (4/294) /content/FaceMask32.mp4: 384x640 4 with_masks, 9.2ms
video 1/1 (5/294) /content/Face

In [44]:
!python /content/yolov5/detect.py --source /content/Data2/test/images/maksssksksss627_png.rf.0064c09ae57ba5f8fc460743df837cbc.jpg --weights '/content/yolov5/runs/train/exp5/weights/best.pt' --img 416 --conf 0.5 --save-txt

detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/Data2/test/images/maksssksksss627_png.rf.0064c09ae57ba5f8fc460743df837cbc.jpg, data=yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-99-g3cd66b1 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
new_train_yaml summary: 232 layers, 7251912 parameters, 0 gradients
image 1/1 /content/Data2/test/images/maksssksksss627_png.rf.0064c09ae57ba5f8fc460743df837cbc.jpg: 256x416 1 mask_weared_incorrect, 7 with_masks, 3 without_masks, 12.0ms
Speed: 0.3ms pre-process, 12.0ms inference, 1.2ms NMS per image at shape (1, 3, 416, 4